In [1]:
!pip install pytesseract

In [2]:
!pip install opencv-python

In [3]:
!pip install pymupdf

In [53]:
import os
import cv2
import numpy as np
import pandas as pd
import fitz
import pytesseract
from PIL import Image

In [54]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [55]:
os.environ['PATH'] += r';C:\Program Files\Tesseract-OCR'

Проверка качества работы tesseract, просто наглядная

In [56]:
# Test image upload
image_path = 'image.png'
image = Image.open(image_path)

# Russian language support
custom_config = r'--oem 1 --psm 1 -l rus'  # rus - код языка русского

# Direct version of extraction
table_text = pytesseract.image_to_string(image, config=custom_config)
print(table_text)

Облучаемая Коэффициент риска Коэффициент риска Сумма,
группа злокачественных наследственных х10-2 Зв-1
населения новообразований, эффектов,
х10-2 Зв-1 х10-2 Зв-1
Все население 5,5 0,2 5,7
Взрослые 4,1 0,1 4,2




In [57]:
# Split to rows
rows = table_text.split('\n')

# Split to elements
table_data = [row.split('\t') for row in rows]

# Make a dataframe
df = pd.DataFrame(table_data)
print(df)

                                                   0
0  Облучаемая Коэффициент риска Коэффициент риска...
1   группа злокачественных наследственных х10-2 Зв-1
2               населения новообразований, эффектов,
3                              х10-2 Зв-1 х10-2 Зв-1
4                          Все население 5,5 0,2 5,7
5                               Взрослые 4,1 0,1 4,2
6                                                   
7                                                   


В результате все строки оказались в разный ячейках, хотя ячейки разделялись по другому принципу

Обобщим обработку на случай pdf файла на входе, сделаем пердобработку и выполним нарезку

In [58]:
# In case of .pdf file
pages = []
file = 'test.pdf'
doc = fitz.open(file)
for n in range(doc.page_count - 1):
    page = doc.load_page(n)
    pix = page.get_pixmap()
    image = np.frombuffer(pix.samples,
                          dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
    image = np.ascontiguousarray(image[..., [2, 1, 0]])
    pages.append(image)

In [59]:
# Remove color info
gray_image = image[:, :, 0]
print(type(gray_image))

<class 'numpy.ndarray'>


In [60]:
# Thresholding image, ret = value, thresh_value = new binary image
ret, thresh_value = cv2.threshold(gray_image, 180, 255, cv2.THRESH_BINARY_INV)

In [61]:
# Dilating image some kind of binary blur
kernel = np.ones((3, 3), np.uint8)
dilated_value = cv2.dilate(thresh_value, kernel, iterations=1)

In [62]:
# Looking for contours, get list of contours and its hierarchy structure
contours, hierarchy = cv2.findContours(dilated_value, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [63]:
# Extracting coordinates and filtering them empirically
coordinates = []  # our target list of contours
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)  # return bounding box of contour
    if (h > 20 or w > 20) and (w / h > 2 or h / w > 2):
        coordinates.append((x, y, w, h))

coordinates

[(87, 82, 513, 176),
 (89, 225, 283, 31),
 (288, 232, 39, 10),
 (226, 232, 60, 10),
 (204, 232, 21, 8),
 (133, 232, 70, 10),
 (94, 229, 37, 13),
 (518, 209, 57, 15),
 (470, 209, 46, 15),
 (421, 209, 47, 15),
 (373, 209, 47, 15),
 (89, 209, 283, 15),
 (94, 212, 86, 10),
 (518, 193, 57, 15),
 (470, 193, 46, 15),
 (421, 193, 47, 15),
 (373, 193, 47, 15),
 (89, 193, 283, 15),
 (518, 176, 57, 16),
 (470, 176, 46, 16),
 (421, 176, 47, 16),
 (373, 176, 47, 16),
 (89, 176, 283, 16),
 (94, 180, 57, 10),
 (518, 160, 57, 15),
 (470, 160, 46, 15),
 (421, 160, 47, 15),
 (373, 160, 47, 15),
 (89, 160, 283, 15),
 (518, 144, 57, 15),
 (470, 144, 46, 15),
 (421, 144, 47, 15),
 (373, 144, 47, 15),
 (89, 144, 283, 15),
 (518, 128, 57, 15),
 (470, 128, 46, 15),
 (421, 128, 47, 15),
 (373, 128, 47, 15),
 (89, 128, 283, 15),
 (110, 120, 33, 7),
 (518, 111, 57, 16),
 (535, 114, 25, 11),
 (470, 111, 46, 16),
 (481, 114, 25, 11),
 (421, 111, 47, 16),
 (433, 114, 26, 11),
 (373, 111, 47, 16),
 (385, 114, 26, 11

Идея удалить символы с картинки чтобы openCV было проще находить прямые. В результате научился находить прямые через tesseract

In [64]:
# Thresholding image, ret = value, thresh_value = new binary image
ret, thresh_value = cv2.threshold(gray_image, 180, 255, cv2.THRESH_BINARY_INV)

In [65]:
# Looking for contours, get list of contours and its hierarchy structure
contours, hierarchy = cv2.findContours(dilated_value, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Create a mask image for the characters
mask = np.ones_like(gray_image, dtype=np.uint8) * 255

# Extracting coordinates and filtering them empirically
coordinates = []  # our target list of contours
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)  # return bounding box of contour
    if h < 100 and w < 100:
        coordinates.append((x, y, w, h))

coordinates

[(594, 218, 5, 3),
 (594, 186, 5, 3),
 (590, 132, 10, 9),
 (588, 114, 12, 11),
 (564, 253, 3, 3),
 (505, 252, 5, 4),
 (457, 252, 5, 4),
 (567, 248, 3, 3),
 (561, 248, 3, 3),
 (138, 248, 3, 3),
 (131, 248, 6, 3),
 (518, 225, 57, 31),
 (470, 225, 46, 31),
 (421, 225, 47, 31),
 (373, 225, 47, 31),
 (288, 232, 39, 10),
 (294, 237, 3, 3),
 (226, 232, 60, 10),
 (263, 234, 6, 3),
 (204, 232, 21, 8),
 (133, 232, 70, 10),
 (158, 235, 3, 3),
 (94, 229, 37, 13),
 (100, 234, 4, 4),
 (564, 221, 3, 3),
 (505, 221, 3, 3),
 (567, 216, 3, 3),
 (561, 216, 3, 3),
 (552, 216, 3, 3),
 (508, 216, 3, 3),
 (502, 216, 3, 3),
 (518, 209, 57, 15),
 (470, 209, 46, 15),
 (421, 209, 47, 15),
 (373, 209, 47, 15),
 (94, 212, 86, 10),
 (165, 217, 3, 3),
 (161, 217, 3, 3),
 (145, 217, 4, 3),
 (134, 217, 3, 3),
 (564, 205, 3, 3),
 (409, 205, 3, 3),
 (281, 205, 3, 3),
 (212, 205, 3, 3),
 (190, 205, 3, 3),
 (172, 205, 3, 3),
 (97, 205, 5, 3),
 (504, 204, 3, 4),
 (258, 204, 4, 4),
 (145, 203, 4, 5),
 (260, 202, 3, 3),
 (16

In [66]:
# Remove color info
gray_image = image[:, :, 0].astype(np.uint8)

# Перебор координат контуров
for (x, y, w, h) in coordinates:
    # Отрисовка прямоугольника вокруг контура
    cv2.rectangle(gray_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Отображение изображения с нарисованными прямоугольниками
cv2.imshow('Image with Lines', gray_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [67]:
# Загрузка изображения
image = cv2.imread('image.png')

# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Адаптивная бинаризация для выделения текста
binary_image = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

# Фильтрация контуров для удаления шумов
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
filtered_contours = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    if w > 10 and h > 10:  # Фильтрация маленьких контуров
        filtered_contours.append(contour)

print(filtered_contours)

# Создание маски для удаления текста
mask = np.ones(gray_image.shape, dtype=np.uint8) * 255
cv2.drawContours(mask, filtered_contours, -1, 0, thickness=cv2.FILLED)

# Применение маски к исходному изображению
result = cv2.bitwise_and(image, image, mask=mask)

# Отображение результата
cv2.imshow('Результат', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

[array([[[  8,   6]],

       [[  8, 176]],

       [[839, 176]],

       [[839,   6]]], dtype=int32)]


In [68]:
# Открытие PDF-файла
file = 'image.pdf'
doc = fitz.open(file)

# Перебор страниц PDF-файла
for page_num in range(len(doc)):
    page = doc[page_num]

    # Получение списка блоков текста на странице
    blocks = page.get_text("blocks")

    # Удаление текста из блоков
    for block in blocks:
        if block[4] == 0:
            # Установка пустого текста для блока
            block[4] = ""

    # Получение списка изображений на странице
    images = page.get_page_image_blocks()

    # Удаление изображений на странице
    for image in images:
        xref = image[0]
        page.delete_image(xref)

    # Сохранение изменений в странице
    page.update_page()

# Сохранение изменений в PDF-файле
output_file = 'квитанция_no_text.pdf'
doc.save(output_file)
doc.close()

AttributeError: 'Page' object has no attribute 'get_page_image_blocks'

In [321]:
# Загрузка изображения
image = cv2.imread('image.png')

# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Распознавание текста с помощью Tesseract
text = pytesseract.image_to_string(gray_image, lang='rus')

# Получение ограничивающих рамок символов
boxes = pytesseract.image_to_boxes(gray_image, lang='rus')

# # Вывод распознанного текста
# print("Распознанный текст:")
# print(text)

# Разделение строки на список строк
box_lines = boxes.strip().split('\n')

# Преобразование каждой строки в список
boxes_list = [line.split() for line in box_lines]

# print(boxes_list)

# Так как cv2 и tesseract используют разные системы координат
height, width, _ = image.shape

# Список границ таблицы
borders = []

# Вывод ограничивающих рамок символов
for box in boxes_list:
    if box[0] == '~':
        x, y, w, h = int(box[1]), int(box[2]), int(box[3]), int(box[4])
        borders.append([x, y, w, h])
        # cv2.rectangle(image, (x, height - y), (w, height - h), (0, 255, 255), 1)

# for box in boxes.splitlines():
#     box = box.split(' ')
#     x, y, w, h = int(box[1]), int(box[2]), int(box[3]), int(box[4])
#     cv2.rectangle(image, (x, height - y), (w, height - h), (0, 0, 255), 1)

# Отображение изображения с ограничивающими рамками символов
cv2.imshow('Image with Character Boxes', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Удалось добиться нахождения линий в таблице, нужно понять как можно кропить изображение исходя из этого

Попробуем найти координаты всех узлов таблицы, по этим узлам восстановим прямоугольники ячеек

In [322]:
borders

[[8, 178, 840, 179],
 [8, 65, 840, 66],
 [8, 37, 840, 38],
 [8, 8, 9, 179],
 [177, 8, 178, 179],
 [424, 8, 425, 179],
 [670, 8, 671, 179],
 [8, 8, 840, 9],
 [839, 8, 840, 179]]

In [323]:
import itertools

# Находим все вертикальные и горизонтальные линии каркаса таблицы
vertical_lines = []
horizontal_lines = []

# Параметры допуска при котором считаем что линии ровные и пересекаются
epsilon = 3

for x1, y1, x2, y2 in borders:
    if abs(x1 - x2) < epsilon:  # Вертикальная линия
        vertical_lines.append((x1, y1, x2, y2))
    elif abs(y1 - y2) < epsilon:  # Горизонтальная линия
        horizontal_lines.append((x1, y1, x2, y2))

# Находим координаты узлов таблицы
table_nodes = []
for v_line, h_line in itertools.product(vertical_lines, horizontal_lines):
    x, y = v_line[0], h_line[1]
    cv2.rectangle(image, (x, height - y), (x + 3, height - y + 3), (0, 0, 255), 1)
    table_nodes.append((x, y))

# Вывод результатов
print("Координаты узлов таблицы:")
for node in table_nodes:
    print(node)

# print("Координаты ячеек таблицы:")
# for cell in table_cells:
#     print(cell)

# Отображение изображения
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Координаты узлов таблицы:
(8, 178)
(8, 65)
(8, 37)
(8, 8)
(177, 178)
(177, 65)
(177, 37)
(177, 8)
(424, 178)
(424, 65)
(424, 37)
(424, 8)
(670, 178)
(670, 65)
(670, 37)
(670, 8)
(839, 178)
(839, 65)
(839, 37)
(839, 8)


In [324]:
# Нахождение количества уникальных значений координат x и y
unique_x = set(x for x, y in table_nodes)
unique_y = set(y for x, y in table_nodes)

# Определение числа строк и столбцов таблицы
num_rows = len(unique_y) - 1
num_columns = len(unique_x) - 1

# Отсортируем узлы по оси x
nodes_sorted_x = sorted(table_nodes, key=lambda x: (x[0], x[1]))

# Создадим список прямоугольников
rectangles = []

# Пройдем по узлам и создадим прямоугольники
for i in range(len(nodes_sorted_x) - 1):
    current_node = nodes_sorted_x[i]
    if nodes_sorted_x[i+1][0] == current_node[0]:
        next_y_node = nodes_sorted_x[i+1]
    else:
        continue
    # Найдем узлы следующие по оси y для текущих узлов
    next_x_nodes = [node for node in nodes_sorted_x if node[1] == current_node[1] and node[0] > current_node[0]]
    if next_x_nodes:
        next_x_node = min(next_x_nodes, key=lambda x: x[0])
        opposite_node = (next_x_node[0], next_y_node[1])
        # Построим прямоугольник по найденным узлам
        rectangle = ((current_node[0], current_node[1]), (opposite_node[0], opposite_node[1]))
        rectangles.append(rectangle)
        cv2.rectangle(image, (current_node[0] + 2, height - current_node[1] + 2), (opposite_node[0] - 2, height - opposite_node[1] - 2), (255, 255, 0), 1)
    else:
        continue
# Выведем список прямоугольников
print(rectangles)

# Отображение изображения
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[((8, 8), (177, 37)), ((8, 37), (177, 65)), ((8, 65), (177, 178)), ((177, 8), (424, 37)), ((177, 37), (424, 65)), ((177, 65), (424, 178)), ((424, 8), (670, 37)), ((424, 37), (670, 65)), ((424, 65), (670, 178)), ((670, 8), (839, 37)), ((670, 37), (839, 65)), ((670, 65), (839, 178))]


In [325]:
rectangles = sorted(rectangles, key=lambda x: (- x[0][1], x[0][0]))
print(rectangles)

[((8, 65), (177, 178)), ((177, 65), (424, 178)), ((424, 65), (670, 178)), ((670, 65), (839, 178)), ((8, 37), (177, 65)), ((177, 37), (424, 65)), ((424, 37), (670, 65)), ((670, 37), (839, 65)), ((8, 8), (177, 37)), ((177, 8), (424, 37)), ((424, 8), (670, 37)), ((670, 8), (839, 37))]


In [326]:
# Создание пустого DataFrame для хранения данных таблицы
table_data = [['' for _ in range(num_columns)] for _ in range(num_rows)]

# Обход прямоугольников ячеек таблицы
for i, rectangle in enumerate(rectangles):
    # Извлечение координат углов прямоугольника
    x1, y1 = rectangle[0]
    x2, y2 = rectangle[1]

    # Обрезание изображения по границам ячейки
    cell_image = image[(height - y2):(height - y1), x1:x2]

    # Russian
    custom_config = r'--oem 1 --psm 1 -l rus'

    # Распознавание текста с помощью Tesseract
    text = pytesseract.image_to_string(cell_image, config=custom_config).replace('\n', ' ')

    # Определение положения ячейки в таблице
    row = i // num_columns
    column = i % num_columns

    # Добавление данных ячейки
    table_data[row][column] = text

# Вывод полученной таблицы
df = pd.DataFrame(table_data)
# Сохранение DataFrame в формате CSV
df.to_csv('table.csv', index=False, header=False)

Попробуем реализовать использование предобученной модели Faster R-CNN из библиотеки PyTorch

In [118]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117
     ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
      --------------------------------------- 0.1/2.5 MB 1.1 MB/s eta 0:00:03
     - -------------------------------------- 0.1/2.5 MB 1.3 MB/s eta 0:00:02
     --- ------------------------------------ 0.2/2.5 MB 1.4 MB/s eta 0:00:02
     ---- ----------------------------------- 0.3/2.5 MB 1.6 MB/s eta 0:00:02
     ------ --------------------------------- 0.4/2.5 MB 1.5 MB/s eta 0:00:02
     --------- ------------------------------ 0.6/2.5 MB 1.6 MB/s eta 0:00:02
     ------------ --------------------------- 0.8/2.5 MB 1.9 MB/s eta 0:00:01
     ----------------- ---------------------- 1.1/2.5 MB 2.4 MB/s eta 0:00:01
     ----------------------- ---------------- 1.5/2.5 MB 3.0 MB/s eta 0:00:01
     -------------------------- ------------- 1.6/2.5 MB 3.3 MB/s eta 0:00:01
     --------------------------- ------------ 1.7/2.5 MB 3.1 MB/s eta 0:00:01
     -------

In [119]:
!pip install torchvision

In [121]:
import torch
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn

In [318]:
# Загрузка предобученной модели Faster R-CNN
model = fasterrcnn_resnet50_fpn(pretrained=True)

# Перевод модели в режим оценки
model.eval()

# Загрузка изображения с таблицей
image = Image.open('eye.jpg')

# Преобразование изображения в тензор
transform = T.ToTensor()
image_tensor = transform(image)

# Добавление размерности пакета (batch dimension)
image_tensor = image_tensor.unsqueeze(0)

# Передача изображения через модель для обнаружения объектов
with torch.no_grad():
    predictions = model(image_tensor)

# Получение результатов обнаружения
boxes = predictions[0]['boxes'].cpu().numpy()  # Координаты ограничивающих прямоугольников ячеек
scores = predictions[0]['scores'].cpu().numpy()  # Уверенность в обнаружении ячеек

print(boxes, '\n', scores)

# Фильтрация результатов по уверенности
confidence_threshold = 0.5
filtered_boxes = boxes[scores >= confidence_threshold]

# Вывод координат ограничивающих прямоугольников ячеек
for box in filtered_boxes:
    x1, y1, x2, y2 = box
    print(f'Координаты ячейки: ({x1}, {y1}) - ({x2}, {y2})')

C:\Users\dsash\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\dsash\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[[  4.290927   3.394928 667.       500.      ]
 [ 75.90003   60.410442 625.97394  367.51984 ]
 [ 63.929653  79.68498  652.075    483.5864  ]
 [194.80998  182.87263  473.1652   307.17676 ]
 [624.4921   253.49231  667.       492.4429  ]
 [  0.         0.       657.91565  492.25204 ]
 [163.79936  174.73889  607.1173   433.4969  ]
 [186.6341   187.74106  493.10522  314.219   ]
 [ 21.250586  21.610126 667.       410.21164 ]
 [187.76561  177.6604   480.0733   306.7911  ]] 
 [0.6331893  0.23897716 0.23883979 0.23732723 0.18757753 0.09228417
 0.06669509 0.05970583 0.05906027 0.05566414]
Координаты ячейки: (4.290926933288574, 3.394927978515625) - (667.0, 500.0)


Пока не решена проблема пустых списков

In [319]:
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.transforms import functional as F

# Загрузка предобученной модели Faster R-CNN
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Задание режима модели в режим вывода
model.eval()

# Загрузка изображения
image = Image.open('eye.jpg')

# Преобразование изображения в тензор
input_tensor = F.to_tensor(image)

# Добавление дополнительной размерности
input_tensor = torch.unsqueeze(input_tensor, 0)

# Получение предсказаний модели
with torch.no_grad():
    predictions = model(input_tensor)

# Обработка предсказаний
boxes = predictions[0]['boxes'].cpu().numpy()
scores = predictions[0]['scores'].cpu().numpy()
labels = predictions[0]['labels'].cpu().numpy()

image = cv2.imread('eye.jpg')
height, width, _ = image.shape

# Вывод результатов
for box, score, label in zip(boxes, scores, labels):
    print('Bounding Box:', box)
    x, y, w, h = [int(x) for x in box]
    print('Confidence Score:', score)
    print('Label:', label)
    print('----------------------')
    cv2.rectangle(image, (x, height -  y), (w, height - h), (0, 0, 255), 1)

cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Bounding Box: [  4.290927   3.394928 667.       500.      ]
Confidence Score: 0.6331893
Label: 65
----------------------
Bounding Box: [ 75.90003   60.410442 625.97394  367.51984 ]
Confidence Score: 0.23897716
Label: 17
----------------------
Bounding Box: [ 63.929653  79.68498  652.075    483.5864  ]
Confidence Score: 0.23883979
Label: 1
----------------------
Bounding Box: [194.80998 182.87263 473.1652  307.17676]
Confidence Score: 0.23732723
Label: 17
----------------------
Bounding Box: [624.4921  253.49231 667.      492.4429 ]
Confidence Score: 0.18757753
Label: 1
----------------------
Bounding Box: [  0.        0.      657.91565 492.25204]
Confidence Score: 0.09228417
Label: 3
----------------------
Bounding Box: [163.79936 174.73889 607.1173  433.4969 ]
Confidence Score: 0.06669509
Label: 1
----------------------
Bounding Box: [186.6341  187.74106 493.10522 314.219  ]
Confidence Score: 0.059705827
Label: 1
----------------------
Bounding Box: [ 21.250586  21.610126 667.       4

По пока не понятным мне причинам Faster R-CNN хорошо распознает объекты реального мира, но не воспринимает таблицы

In [176]:
import torch
import torchvision.transforms as transforms
from torchvision.models.detection import ssdlite320_mobilenet_v3_large

# Загрузка предобученной модели SSD
model = ssdlite320_mobilenet_v3_large(pretrained=True)

# Установка режима оценки модели
model.eval()

# Загрузка изображения
image = Image.open('test1.jpg')

# Преобразование изображения в тензор и нормализация
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
image_tensor = transform(image)
image_tensor = image_tensor.unsqueeze(0)  # Добавление размерности пакета

# Предсказание с помощью модели
with torch.no_grad():
    outputs = model(image_tensor)

# Извлечение координат ограничивающих рамок и вероятностей объектов
boxes = outputs[0]['boxes'].numpy()
scores = outputs[0]['scores'].numpy()

print(boxes)

image = cv2.imread('test1.jpg')

# Вывод обнаруженных объектов с порогом вероятности
threshold = 0.1
for box, score in zip(boxes, scores):
    if score > threshold:
        x, y, w, h = [int(x) for x in box]
        cv2.rectangle(image, (x, y), (w, h), (0, 0, 255), 1)

# Отображение изображения с обнаруженными объектами
cv2.imshow('Image with Objects', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\dsash\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSDLite320_MobileNet_V3_Large_Weights.COCO_V1`. You can also use `weights=SSDLite320_MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[[  9.487405  20.090921 664.7107   501.00116 ]
 [ 11.051949   9.414893 674.3935   518.10156 ]
 [381.91617  418.6652   413.17334  451.0445  ]
 ...
 [188.25041  100.843376 280.07553  157.37564 ]
 [411.09344  414.08173  532.382    456.8201  ]
 [347.21545   46.968636 509.9813    95.19943 ]]


SSD так же не справилась с нахождением ячеек таблицы

In [164]:
!pip install pydarknet

ERROR: Could not find a version that satisfies the requirement pydarknet (from versions: none)
ERROR: No matching distribution found for pydarknet


In [163]:
import cv2
from pydarknet import Detector, Image

# Загрузка конфигурационного файла и весов модели YOLO
config_file = "yolov4.cfg"
weight_file = "yolov4.weights"
data_file = "coco.data"

# Создание экземпляра детектора YOLO
net = Detector(bytes(config_file, encoding="utf-8"),
               bytes(weight_file, encoding="utf-8"),
               0,
               bytes(data_file, encoding="utf-8"))

# Загрузка изображения
image = cv2.imread('image.jpg')

# Преобразование изображения в формат, поддерживаемый YOLO
darknet_image = Image(image)

# Обнаружение объектов на изображении
results = net.detect(darknet_image)

# Итерация по результатам обнаружения
for cat, score, bounds in results:
    x, y, w, h = bounds

    # Отрисовка ограничивающей рамки и метки объекта
    cv2.rectangle(image, (int(x - w/2), int(y - h/2)), (int(x + w/2), int(y + h/2)), (0, 255, 0), 2)
    cv2.putText(image, cat.decode("utf-8"), (int(x - w/2), int(y - h/2 - 10)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Отображение изображения с обнаруженными объектами
cv2.imshow('Image with Objects', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'pydarknet'

YOLO заставить работать не получилось

In [178]:
import torch
import torchvision
from torchvision.models.detection import retinanet
from torchvision.transforms import functional as F
import cv2

# Загрузка предобученной модели RetinaNet
model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True)
model.eval()

# Загрузка классов объектов
class_labels = ["class_0", "class_1", "class_2"]  # Замените на свои классы объектов

# Загрузка изображения
image = cv2.imread('test1.jpg')
height, width, _ = image.shape

image_tensor = F.to_tensor(image)

# Предобработка изображения
input_image = image_tensor.unsqueeze(0)

# Получение предсказаний модели
with torch.no_grad():
    prediction = model(input_image)

# Извлечение координат, меток классов и вероятностей
boxes = prediction[0]['boxes']
print(boxes)
labels = prediction[0]['labels']
scores = prediction[0]['scores']

# Отрисовка ограничивающих рамок и меток
for box in boxes:
    x1, y1, x2, y2 = box.tolist()
    cv2.rectangle(image, (int(x1), height - int(y1)), (int(x2), height - int(y2)), (0, 255, 0), 2)

# Отображение изображения с обнаруженными объектами
cv2.imshow('Image with Objects', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

tensor([[339.0235, 112.6919, 629.4863, 470.6991],
        [ 84.0650,  71.8075, 638.3114, 481.7713],
        [337.6313, 162.4893, 643.6937, 472.9701]])


RetinaNet находит какие-то объекты, но не корректно

Из теста этих нейронных сетей можно сделать вывод что с таблицами они работают плохо и требуют значительного дообучения

Для выделения таблицы из корпуса текста попробуем использовать cv2 с такими настройками

In [332]:
import cv2
import numpy as np

# Загрузка изображения
image = cv2.imread('in_context.jpg')

# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Применение порогового преобразования для бинаризации изображения
ret, threshold_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Применение морфологического преобразования для удаления шума и замыкания линий
kernel = np.ones((5, 5), np.uint8)
morph_image = cv2.morphologyEx(threshold_image, cv2.MORPH_CLOSE, kernel)

# Нахождение контуров на изображении
contours, hierarchy = cv2.findContours(morph_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Создание списка координат ограничивающих рамок ячеек
cell_boxes = []

# Фильтрация контуров и определение ограничивающих рамок ячеек
for contour in contours:
    # Определение площади контура
    area = cv2.contourArea(contour)

    # Фильтрация контуров по площади
    if area > 3000:
        # Определение ограничивающей рамки контура
        x, y, w, h = cv2.boundingRect(contour)

        # Добавление ограничивающей рамки в список
        cell_boxes.append((x, y, x + w, y + h))

# Вывод координат ограничивающих рамок ячеек
for box in cell_boxes:
    x1, y1, x2, y2 = box
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Отображение изображения с выделенными ячейками
cv2.imshow('Image with Cells', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Зная координаты всех букв можно попытаться их кластеризовать. Попробуем DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

# Предположим, что у нас есть список координат всех букв на изображении
letter_coordinates = []

# Создаем экземпляр алгоритма DBSCAN
dbscan = DBSCAN(eps=10, min_samples=2)  # Параметры eps и min_samples нужно настроить под конкретную задачу

# Применяем алгоритм DBSCAN к координатам букв
clusters = dbscan.fit_predict(letter_coordinates)

# Выводим результаты
for label in set(clusters):
    cluster_points = [coord for coord, cluster_label in zip(letter_coordinates, clusters) if cluster_label == label]
    print(f"Cluster {label}: {cluster_points}")

Алгоритмы кластеризации типа DBSCAN или Ланса-Уильямса дают плохую точность из-за неоднородной плотности таблиц

Самое первое решение через нарезку изображения

In [82]:
# Remove color info
gray_image = image[:, :, 0].astype(np.uint8)

# Перебор координат контуров
for (x, y, w, h) in coordinates:
    # Отрисовка прямоугольника вокруг контура
    cv2.rectangle(gray_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Отображение изображения с нарисованными прямоугольниками
cv2.imshow('Image with Lines', gray_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [57]:
# Thresholding image, ret = value, thresh_value = new binary image
ret, thresh_value = cv2.threshold(gray_image, 180, 255, cv2.THRESH_BINARY_INV)

# Dilating image some kind of binary blur
kernel = np.ones((2, 2), np.uint8)
dilated_value = cv2.dilate(thresh_value, kernel, iterations=2)

In [58]:
cv2.imshow('Image with Lines', dilated_value)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [226]:
recognized_table = row = []
prev_y = 0
coordinates.sort()  #sort by x
coordinates.sort(key=lambda x: x[1])  # sort by y
for coord in coordinates:
    x, y, w, h = coord
    if y > prev_y + 100:  #new row if y is changed
        recognized_table.append(row)
        row = []
    crop_img = image[y:y + h, x:x + w]
    custom_config = r'--oem 1 --psm 1 -l rus'  # rus - код языка русского
    recognized_string = pytesseract.image_to_string(crop_img, config=custom_config)
    row.append(recognized_string.replace("\n", " "))
    prev_y = y

recognized_table

['Облучаемая | Коэффициент риска | Коэффициент риска ‘Сумма, группа злокачественных наследственных х10-2 Зв-1 населения новообразований, эффектов, х10-2 38-1 х10-2 Зв-1 Все население. 55 02 57 Взрослые 41 ол 4.2  ',
 "'Облучаемая  группа населения  ",
 'Коэффициент риска злокачественных новообразований,  х10-2 Зв-1  ',
 'Коэффициент риска  наследственных  эффектов, х10-2 Зв-1  ',
 '‘Сумма, х10-2 Зв-1  ',
 'Все население ',
 '',
 '',
 '',
 'Взрослые ',
 '',
 '0,1 ',
 '']

Сгенерируем датасет для оценки качества выполнения задачи. Возможно дообучим pytorch nn. Надо доработать генератор с учетом специфики конкретных таблиц ГОСТа

In [1]:
!pip install tabula - py

     ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.0 MB 1.3 MB/s eta 0:00:10
     ---------------------------------------- 0.1/12.0 MB 1.3 MB/s eta 0:00:10
      --------------------------------------- 0.2/12.0 MB 1.8 MB/s eta 0:00:07
      --------------------------------------- 0.3/12.0 MB 1.7 MB/s eta 0:00:07
     - -------------------------------------- 0.4/12.0 MB 1.9 MB/s eta 0:00:07
     - -------------------------------------- 0.5/12.0 MB 1.7 MB/s eta 0:00:07
     - -------------------------------------- 0.6/12.0 MB 1.7 MB/s eta 0:00:07
     -- ------------------------------------- 0.7/12.0 MB 1.9 MB/s eta 0:00:07
     --- ------------------------------------ 0.9/12.0 MB 2.2 MB/s eta 0:00:06
     --- ------------------------------------ 1.1/12.0 MB 2.2 MB/s eta 0:00:06
     --- ------------------------------------ 1.2/12.0 MB 2.2 MB/s eta 0:00:05
     ---- ----------------------------------- 1.3/12.0 MB 2

In [3]:
!pip install fpdf

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40713 sha256=3d030d429e9eb0115ed48106e615e2162d4ee5f59fce765845635860f679b694
  Stored in directory: c:\users\dsash\appdata\local\pip\cache\wheels\44\35\8b\86ce00cec7e4d13c5f189680ae0fa82f919bedc066c2cddae9
Successfully built fpdf
Note: you may need to restart the kernel to use updated packages.


In [18]:
import tabula
import pandas as pd
import random
import string
from fpdf import FPDF

# Функция для генерации случайного текста или цифр
def generate_random_data(length):
    data_type = random.choice(["letters", "digits"])
    if data_type == "digits":
        return ''.join(random.choice(string.digits) for _ in range(length))
    else:
        return ''.join(random.choice(string.ascii_letters) for _ in range(length))

# Функция для генерации случайной таблицы
def generate_random_table(num_rows, num_cols):
    table_data = []
    for _ in range(num_rows):
        row = [generate_random_data(random.randint(1, 5)) for _ in range(num_cols)]
        table_data.append(row)
    return table_data

# Функция для создания датасета
def generate_dataset(num_samples, rows, cols):
    # Создание директории "dataset", если она не существует
    directory = "dataset"
    if os.path.exists(directory):
        for file_name in os.listdir(directory):
            file_path = os.path.join(directory, file_name)
            os.remove(file_path)
    else:
        os.makedirs(directory)

    for i in range(num_samples):
        # Генерация случайной таблицы
        table_data = generate_random_table(rows, cols)
        df = pd.DataFrame(table_data)

        # Создание PDF файла с таблицей
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)

        for row in table_data:
            for item in row:
                pdf.cell(30, 10, str(item), border=1)
            pdf.ln()

        pdf_file = f"dataset/sample_table_{i}.pdf"
        pdf.output(pdf_file)

        # Создание CSV файла с меткой
        csv_file = f"dataset/sample_table"
        df.to_csv(f"{csv_file}_{i}.csv", index=False)

# Параметры генерации датасета
num_samples = 10  # Количество образцов
rows = 5  # Количество строк в таблице
cols = 5  # Количество столбцов в таблице

# Создание датасета
generate_dataset(num_samples, rows, cols)

Добавить в датасете таблицы разных размеров, шумы и искажения перспективы на изображениях